### 앙상블 학습
* voting : 전체데이터를 각 알고리즘에 다 넣어보고 결과를 알려주는데 하나만 정하면 hard vothing, 나온결 평균내면 soft voting
* bagging : 테이터를 데이터 개수 만큼 복원추출로 샘플을 생성해서(=boostrap) 이를 알고리즘에 집계(평균 or 가중평균)
* boosting : bagging알고리즘 중 취약한 곳에 가중치를 두어 다시 알고리즘으로 돌림 ex.시계열 데이터에 적합. 순서가 있기 때문


* sklearn은 보팅 방식의 앙상블을 구현한 VotingClassifier 클래스를 제공하고 있음
* 보딩 방힛ㄱ의 앙상블을 이용해 위스콘신 유방암 데이터 세트를 예측 분석해 보겠음.
* 위스콘신 유방암 데이터 세트는 유방암의 악성종양, 양성종양 여부를 결정하는 이진 분류 데이터 세트이며 종앙의 크기, 모양 등의 형태와 관련 많은 피쳐가 있음
*로지스틱 회귀와 KNN기반으로 보팅 분류기를 생성
*사이킷런은 VotingClassifier 클래스를 이용해 보팅 분류기를 생성 할 수 있음


In [1]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns = cancer.feature_names)
data_df.head(3)



,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758


In [3]:
#VotingClassifier의 주요 인자 : estimators와 voting 값
#estimator : 리스트의 값으로 보팅에 사용될 여러개의 classifier 객체들을 튜플형식으로 입력 받아 hard,soft 선택

#개별 모델은 로지스틱 회귀와 KNN
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors= 8)

#개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기 
vo_clf = VotingClassifier(estimators = [('LR', lr_clf), ('kNN', knn_clf)], voting  = 'soft')
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target , test_size = 0.2, random_state= 5)

#VotingClassifier 학습/예측/평가
vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)

print(f"Voting분류기 정확도 : {accuracy_score(y_test, pred)}")


Voting분류기 정확도 : 0.9473684210526315


C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
#개별 모델의 학습/예측/평가
classifiers = [lr_clf, knn_clf]

for classifier in classifiers:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    class_name = classifier.__class__.__name__
    print('{0} 정확도 : {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))

LogisticRegression 정확도 : 0.9561
KNeighborsClassifier 정확도 : 0.9561


C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
classifiers

[LogisticRegression(), KNeighborsClassifier(n_neighbors=8)]